## Covid Clinical Research Webscrping Demo
### Overview

**Task:** Create a table file with the Covid clinical pesearch paper information where the data is in a standardized format that can be easily used in a tool that collects most up to date papers on COVID-19 or other `interested_condtion`

**Link:** https://clinicaltrials.gov/ct2/

**Last update:** October 6, 2021


**Steps of Scraper**
 
1.	`scrape_one_page`  
    A.	Scrapes all the table rows and columns from one page  
    B.	Finds the current row number on the condition results page (i.e. the ` current_row`)  
    C.	Clicks on the next page  
    D.	Returns the data (i.e. ` page_dict `) to ` get_allpage_data `  
1.	`get_allpage_data`  
    A.	Finds and takes the result message text (i.e. “Showing: _ of _ studies”)  
    B.	Assumes each page shows 100 results   
    C.	Get total number of pages using  total_row_int/100     
    D.	Check if len(all_page_dict) == row_count_int   
    E.  Returns the the complete provider data (i.e. ` all_page_dict `) to ` write_out_data`   
1.	`write_out_data`  
    A.	Takes all provider data (i.e. ` all_page_dict `)     
    B.	Convert the condition data from list format to a data frame (i.e. `raw_df`)     
    C.	Write out all condition data as data frame (i.e. `raw_df`)  
    D.	Selects the columns from interested condition a data frame we need from all data (`raw_df`)
    E.  Writes out clean data (i.e. `clean_df`)   
    

In [14]:
!pip list

Package                            Version  
---------------------------------- ---------
alabaster                          0.7.12   
anaconda-client                    1.7.2    
anaconda-navigator                 1.9.7    
anaconda-project                   0.8.2    
argcomplete                        1.12.3   
asn1crypto                         0.24.0   
astroid                            2.2.5    
astropy                            3.1.2    
atomicwrites                       1.3.0    
attrs                              19.1.0   
autopep8                           1.5.7    
Babel                              2.6.0    
backcall                           0.1.0    
backports.entry-points-selectable  1.1.0    
backports.os                       0.1.1    
backports.shutil-get-terminal-size 1.0.0    
beautifulsoup4                     4.7.1    
bitarray                           0.8.3    
bkcharts                           0.2      
bleach                             3.1.0    
bokeh     

In [25]:
#the webmanager auto detect your Chrome version and once installled, 
#Driver will be found in cache and won't need to be installed again
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

import pandas as pd
import re 
import math
import numpy as np
from selenium.webdriver.support.ui import Select
import time
# import git
import logging





====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\FYang\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [28]:
interested_condtion = ["covid"]

# find current time
timestr = time.strftime("%Y%m%d-%H%M")

def create_path(last_part_path, file_type = ".csv"): 
    ''' 
    take last part of path and provide path with identifying information
    '''
    path = root_path + last_part_path + "_" + timestr + "_" + file_type
    
    return path

root_path = "C:/Users/FYang/Projects/DS_NLPSpecialization/webscraping/Webscrping Demo for Oct Hackathon/"

dict_out_path = create_path("interim/page_dict", ".txt")
raw_df_path = create_path("raw/covid_study_df")
logging_path = create_path("interim/log", ".log")
clean_df_path = create_path("cleaned_outputs/condition_study_clean")

# configure logging
logging.basicConfig(filename= logging_path, filemode='w',  
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%d-%b-%y %H:%M:%S',
                    level=logging.INFO)


driver.get("https://clinicaltrials.gov/ct2/")
e=driver.find_element_by_id('home-search-condition-query')
e.send_keys(interested_condtion)
e.send_keys(Keys.ENTER)

result_length_list = Select(driver.find_element_by_name('theDataTable_length'))
result_length_list.select_by_value("100")


In [29]:
def scrape_page(page,driver):
    page_dict = dict()
    
    # get col names list
    for thead in driver.find_elements_by_xpath('//*[@id="theDataTable"]/thead'):
        col_list = [item.text for item in thead.find_elements_by_xpath(".//*[self::th]")]

    # get row values- rows of the table on page
    for row in driver.find_elements_by_xpath('//*[@id="theDataTable"]/tbody/tr'):
        row_list = [item.text for item in row.find_elements_by_xpath(".//*[self::td]")]

        # combine lists to make dic
        row_dict = {k:v for k, v in zip(col_list, row_list)}

        current_row=int(row_dict.get('Row'))
        page_dict[current_row] = row_dict

        current_row = current_row + 1

    page_button = driver.find_element_by_xpath('//a[@aria-controls= "theDataTable"]').click()       
#     page_button=WebDriverWait(driver,100).until(EC.visibility_of_element_located((By.XPATH ,'//a[@aria-controls= "theDataTable"]')))
#     page_button.click()

    return(page_dict)

### Block Test for scraping selected number of pages and write out the result 

In [61]:
# Block Test for get_allpage_data
all_page_dict = dict()


search_result=driver.find_element_by_class_name('dataTables_info')
results_text=search_result.text

total_row_list=re.findall('\d+(?:\.\d+)?,\d+', results_text)
total_row_int=int(total_row_list[0].replace(',', ''))

temp = total_row_int/100 
pages = math.ceil(temp)


for page in range(1, 2):
    page_dict = scrape_page(page, driver)
    all_page_dict.update(page_dict)

# Block Test for writing out scraped one page

col_names_dict={"Row": "Number",
               "Saved": None,
               "Status": "Study_Paticipant_Recruitment_Status", 
               "Study Title": "Study_Title", 
               "Conditions" : "Conditions",
               "Interventions": "Interventions",
               "Locations": "Locations"
              }

col_names_dict_filter = {k: v for k, v in col_names_dict.items() if v is not None}
    
    #write out all_page_dict
with open(dict_out_path, 'w', encoding='utf-8') as f:
    print(all_page_dict, file=f)

raw_df=pd.DataFrame(all_page_dict).T 
    
raw_df.to_csv(raw_df_path, index=False)
logging.info("finished: writing out raw_df")
    
# clean_df -- all data from all providers (as dataframe)
# drop_duplicates -- dropping duplicated rows
clean_df = raw_df[list(col_names_dict_filter.keys())]
clean_df = clean_df.rename(columns= col_names_dict_filter)

clean_df = clean_df.drop_duplicates()

# log number of duplicated rows dropped
count_duplicate = len(raw_df)-len(clean_df)
logging.info('number of duplicate rows removed =' + str(count_duplicate))

clean_df.to_csv(clean_df_path, index=False)

logging.info("finished: writing out clean_df")


In [19]:
all_page_dict

{1: {'Row': '1',
  'Saved': '',
  'Status': 'Enrolling by invitation',
  'Study Title': 'COVID SAFE: COVID-19 Screening Assessment for Exposure',
  'Conditions': 'Covid19',
  'Interventions': 'Behavioral: Saliva-based testing',
  'Locations': 'Penn Medicine\nPhiladelphia, Pennsylvania, United States'},
 2: {'Row': '2',
  'Saved': '',
  'Status': 'Not yet recruiting',
  'Study Title': 'CICERO- A Care Home Study of COVID-19',
  'Conditions': 'COVID',
  'Interventions': 'Diagnostic Test: Q16 testing\nDiagnostic Test: Nasopharyngeal swab and main laboratory',
  'Locations': 'Queen Mary University London\nLondon, United Kingdom'},
 3: {'Row': '3',
  'Saved': '',
  'Status': 'Recruiting',
  'Study Title': 'Dexamethasone for COVID-19',
  'Conditions': 'Covid19',
  'Interventions': 'Drug: Dexamethasone',
  'Locations': 'University of Oklahoma Medical Center\nOklahoma City, Oklahoma, United States'},
 4: {'Row': '4',
  'Saved': '',
  'Status': 'Recruiting',
  'Study Title': 'Duvelisib to Combat

In [56]:
raw_df

,Conditions,Interventions,Locations,Row,Saved,Status,Study Title
1,Covid19,Behavioral: Saliva-based testing,"Penn Medicine\nPhiladelphia, Pennsylvania, Uni...",1,,Enrolling by invitation,COVID SAFE: COVID-19 Screening Assessment for ...
2,COVID,Diagnostic Test: Q16 testing\nDiagnostic Test:...,"Queen Mary University London\nLondon, United K...",2,,Not yet recruiting,CICERO- A Care Home Study of COVID-19
3,Covid19,Drug: Dexamethasone,University of Oklahoma Medical Center\nOklahom...,3,,Recruiting,Dexamethasone for COVID-19
4,COVID-19,Drug: Duvelisib\nProcedure: Peripheral blood d...,"Missouri Baptist Medical Center\nSaint Louis, ...",4,,Recruiting,Duvelisib to Combat COVID-19
5,Covid19,Behavioral: NexJ Connected Wellness\nOther: Us...,,5,,Not yet recruiting,Enhancing COVID Rehabilitation With Technology
6,COVID,,"Mansoura faculty of medicine\nMansoura, Dakahl...",6,,"Active, not recruiting",Post Covid-19 Cardiopulmonary and Immunologica...
7,Covid19,Drug: Infliximab\nDrug: Abatacept\nDrug: Remde...,"Banner University Medical Center\nPhoenix, Ari...",7,,Recruiting,Immune Modulators for Treating COVID-19
8,Covid19,Drug: Placebo\nDrug: Fisetin,"Mayo Clinic in Rochester\nRochester, Minnesota...",8,,Enrolling by invitation,COVID-FISETIN: Pilot in SARS-CoV-2 of Fisetin ...
9,Covid19,Other: Balance assessments,"Duke University Health System\nDurham, North C...",9,,Recruiting,Balance in COVID-19
10,Covid19,Drug: Ivermectin nasal\nDrug: Ivermectin oral\...,,10,,Not yet recruiting,Ivermectin Nasal Spray for COVID19 Patients


In [62]:
clean_df

,Number,Study_Paticipant_Recruitment_Status,Study_Title,Conditions,Interventions,Locations
1,1,Enrolling by invitation,COVID SAFE: COVID-19 Screening Assessment for ...,Covid19,Behavioral: Saliva-based testing,"Penn Medicine\nPhiladelphia, Pennsylvania, Uni..."
2,2,Not yet recruiting,CICERO- A Care Home Study of COVID-19,COVID,Diagnostic Test: Q16 testing\nDiagnostic Test:...,"Queen Mary University London\nLondon, United K..."
3,3,Recruiting,Dexamethasone for COVID-19,Covid19,Drug: Dexamethasone,University of Oklahoma Medical Center\nOklahom...
4,4,Recruiting,Duvelisib to Combat COVID-19,COVID-19,Drug: Duvelisib\nProcedure: Peripheral blood d...,"Missouri Baptist Medical Center\nSaint Louis, ..."
5,5,Not yet recruiting,Enhancing COVID Rehabilitation With Technology,Covid19,Behavioral: NexJ Connected Wellness\nOther: Us...,
6,6,"Active, not recruiting",Post Covid-19 Cardiopulmonary and Immunologica...,COVID,,"Mansoura faculty of medicine\nMansoura, Dakahl..."
7,7,Recruiting,Immune Modulators for Treating COVID-19,Covid19,Drug: Infliximab\nDrug: Abatacept\nDrug: Remde...,"Banner University Medical Center\nPhoenix, Ari..."
8,8,Enrolling by invitation,COVID-FISETIN: Pilot in SARS-CoV-2 of Fisetin ...,Covid19,Drug: Placebo\nDrug: Fisetin,"Mayo Clinic in Rochester\nRochester, Minnesota..."
9,9,Recruiting,Balance in COVID-19,Covid19,Other: Balance assessments,"Duke University Health System\nDurham, North C..."
10,10,Not yet recruiting,Ivermectin Nasal Spray for COVID19 Patients,Covid19,Drug: Ivermectin nasal\nDrug: Ivermectin oral\...,


### Below are for scraping through all pages

In [10]:
def get_allpage_data(pages, driver):
    '''returns all data for the given condition 
    
    Keyword arguments:
    pages -- total number of pages for condition, integers inclusive of last page
    driver -- chrome driver
    '''
    # one_provider_dict -- all data scraped for this condition as dict
    all_page_dict = dict()
    
    
    search_result=driver.find_element_by_class_name('dataTables_info')
    results_text=search_result.text
    
    total_row_list=re.findall('\d+(?:\.\d+)?,\d+', results_text)
    total_row_int=int(total_row_list[0].replace(',', ''))
    
    temp = total_row_int/100 
    pages = math.ceil(temp)

    
    for page in range(1, pages + 1):
        page_dict = scrape_page(page, driver)
        all_page_dict.update(page_dict)
    
    try:
        assert len(all_page_dict) == total_row_int, "number of values scraped does not match number of rows"
    except AssertionError:
        logging.exception("number of values scraped does not match number of rows")
        raise

    logging.info('complete data scraped for ' + str(interested_condtion))

    return(all_page_dict)




In [11]:
def write_out_data(all_page_dict, clean_df_path):
    col_names_dict={"Row": "Number",
               "Saved": None,
               "Status": "Study_Paticipant_Recruitment_Status", 
               "Study Title": "Study_Title", 
               "Conditions" : "Conditions",
               "Interventions": "Interventions",
               "Location": "Location"
              }

    col_names_dict_filter = {k: v for k, v in col_names_dict.items() if v is not None}
    
    #write out all_page_dict
    with open(dict_out_path, 'w', encoding='utf-8') as f:
        print(all_page_dict, file=f)

    raw_df = pd.concat({k: pd.DataFrame(v).T for k, v in all_page_dict.items()}, axis=0)
    
    raw_df.to_csv(raw_df_path, index=False)
    logging.info("finished: writing out raw_df")
    
    # clean_df -- all data from all providers (as dataframe)
    # drop_duplicates -- dropping duplicated rows
    clean_df = raw_df[list(col_names_dict_filter.keys())]
    clean_df = clean_df.rename(columns= col_names_dict_filter)
    
    clean_df = clean_df.drop_duplicates()
    
    # log number of duplicated rows dropped
    count_duplicate = len(raw_df)-len(clean_df)
    logging.info('number of duplicate rows removed =' + str(count_duplicate))
    
    clean_df.to_csv(clean_df_path, index=False)
    
    logging.info("finished: writing out clean_df")
        

In [ ]:
# write out data
write_out_data(all_page_dict,clean_df_path)